In [23]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
import pandas as pd
import csv
import seaborn as sns
sns.set_theme(style="whitegrid")
from tabulate import tabulate
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from tqdm import tqdm  # Import tqdm for the progress bar
import plotly.io as pio
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.dummy import DummyRegressor
from sklearn.metrics import (
    mean_absolute_error as mae,
    r2_score as r2,
    mean_absolute_percentage_error as mape,
    mean_squared_error as mse)
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

In [2]:
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.ensemble import RandomForestClassifier

In [3]:
from feature_engine.selection import DropCorrelatedFeatures

# Import data

In [4]:
file_path = '../train_1.csv'

data = pd.read_csv(file_path)
data

,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,53.588771,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21188,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,43.202659,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21189,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,50.148287,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21190,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,5.405448,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21191,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,6.249958,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [5]:
close_corr =data.corr(method = 'pearson')
close_corr

,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
mean_atomic_mass,1.000000,0.815443,0.940210,0.745262,-0.104175,-0.098096,0.125842,0.446325,0.196643,0.130723,...,0.305133,0.392666,0.321934,-0.156998,-0.146226,-0.107039,0.170750,-0.079766,-0.081024,-0.113162
wtd_mean_atomic_mass,0.815443,1.000000,0.847636,0.964012,-0.308124,-0.413358,-0.143792,0.717191,-0.060526,-0.089369,...,0.545974,0.539945,0.549390,-0.375807,-0.331365,-0.038183,0.332698,-0.002807,0.077983,-0.311954
gmean_atomic_mass,0.940210,0.847636,1.000000,0.856449,-0.189982,-0.232456,-0.175801,0.458534,-0.121735,-0.166169,...,0.428030,0.511756,0.450532,-0.306164,-0.308083,-0.164446,0.274154,-0.124097,-0.117175,-0.229885
wtd_gmean_atomic_mass,0.745262,0.964012,0.856449,1.000000,-0.370411,-0.485170,-0.351986,0.673826,-0.274412,-0.331811,...,0.614231,0.608412,0.623435,-0.477714,-0.448308,-0.077587,0.411259,-0.032439,0.030924,-0.369429
entropy_atomic_mass,-0.104175,-0.308124,-0.189982,-0.370411,1.000000,0.889471,0.537854,-0.285161,0.356920,0.412836,...,-0.585717,-0.553177,-0.591685,0.963558,0.861150,0.202634,-0.349133,0.087673,0.018792,0.542854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
range_Valence,-0.107039,-0.038183,-0.164446,-0.077587,0.202634,0.160734,0.208458,0.023641,0.192080,0.177327,...,0.113189,-0.039978,0.034796,0.154607,0.203963,1.000000,0.114614,0.973837,0.867507,-0.144641
wtd_range_Valence,0.170750,0.332698,0.274154,0.411259,-0.349133,-0.546303,-0.363967,0.595038,-0.271879,-0.389699,...,0.684399,0.533512,0.682172,-0.416558,-0.639381,0.114614,1.000000,0.184713,0.132202,-0.439821
std_Valence,-0.079766,-0.002807,-0.124097,-0.032439,0.087673,0.056051,0.134726,0.067124,0.144277,0.123676,...,0.187765,0.035886,0.109815,0.034199,0.089699,0.973837,0.184713,1.000000,0.885844,-0.208882
wtd_std_Valence,-0.081024,0.077983,-0.117175,0.030924,0.018792,-0.006994,0.102883,0.093093,0.122862,0.156034,...,0.259755,0.089327,0.169942,-0.017114,0.081537,0.867507,0.132202,0.885844,1.000000,-0.300651


# Data preparation

In [6]:
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]



X.head()

,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,...,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,...,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059
1,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,...,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606
2,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,...,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697
3,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,...,2.25,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952
4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,53.588771,...,2.25,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809


In [7]:
y.head()

0    29.0
1    26.0
2    19.0
3    22.0
4    23.0
Name: critical_temp, dtype: float64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0,
)

X_train.shape, X_test.shape

((16954, 80), (4239, 80))

## Remove correlated: Feature-engine
#### Smart approach
###### _From each group of correlated variables, we retain the one with the highest importance, derived from a machine learning model._

In [9]:

sel = SmartCorrelatedSelection(
    method="pearson",
    threshold=0.8,
    selection_method='model_performance',
    estimator=RandomForestRegressor(n_estimators=10, random_state=20),
    scoring='r2', 
    cv=3,
)

# fit finds the correlated features
sel.fit(X_train, y_train)




SmartCorrelatedSelection(estimator=RandomForestRegressor(n_estimators=10,
                                                         random_state=20),
                         scoring='r2', selection_method='model_performance')

In [10]:
sel.features_to_drop_

['mean_atomic_mass',
 'wtd_mean_atomic_mass',
 'wtd_gmean_atomic_mass',
 'entropy_atomic_mass',
 'wtd_entropy_atomic_mass',
 'range_atomic_mass',
 'wtd_range_atomic_mass',
 'wtd_std_atomic_mass',
 'mean_fie',
 'wtd_mean_fie',
 'wtd_gmean_fie',
 'entropy_fie',
 'range_fie',
 'std_fie',
 'mean_atomic_radius',
 'wtd_mean_atomic_radius',
 'wtd_gmean_atomic_radius',
 'wtd_entropy_atomic_radius',
 'range_atomic_radius',
 'std_atomic_radius',
 'wtd_std_atomic_radius',
 'wtd_mean_Density',
 'entropy_Density',
 'range_Density',
 'wtd_std_Density',
 'mean_ElectronAffinity',
 'wtd_gmean_ElectronAffinity',
 'entropy_ElectronAffinity',
 'wtd_entropy_ElectronAffinity',
 'range_ElectronAffinity',
 'wtd_std_ElectronAffinity',
 'mean_FusionHeat',
 'wtd_mean_FusionHeat',
 'wtd_gmean_FusionHeat',
 'entropy_FusionHeat',
 'wtd_entropy_FusionHeat',
 'range_FusionHeat',
 'wtd_std_FusionHeat',
 'wtd_mean_ThermalConductivity',
 'wtd_gmean_ThermalConductivity',
 'range_ThermalConductivity',
 'wtd_std_ThermalCon

In [11]:
sel.correlated_feature_sets_

[{'gmean_atomic_mass', 'mean_atomic_mass', 'wtd_mean_atomic_mass'},
 {'wtd_gmean_Density', 'wtd_gmean_atomic_mass', 'wtd_mean_Density'},
 {'entropy_Density',
  'entropy_ElectronAffinity',
  'entropy_FusionHeat',
  'entropy_Valence',
  'entropy_atomic_mass',
  'entropy_atomic_radius',
  'entropy_fie',
  'wtd_entropy_FusionHeat',
  'wtd_entropy_Valence',
  'wtd_entropy_atomic_mass',
  'wtd_entropy_atomic_radius'},
 {'range_atomic_mass', 'std_atomic_mass', 'wtd_std_atomic_mass'},
 {'wtd_range_Density', 'wtd_range_atomic_mass'},
 {'gmean_fie', 'mean_fie'},
 {'wtd_gmean_atomic_radius',
  'wtd_gmean_fie',
  'wtd_mean_atomic_radius',
  'wtd_mean_fie',
  'wtd_std_fie'},
 {'wtd_entropy_ElectronAffinity', 'wtd_entropy_fie'},
 {'gmean_Density',
  'range_atomic_radius',
  'range_fie',
  'std_atomic_radius',
  'std_fie',
  'wtd_std_atomic_radius'},
 {'gmean_atomic_radius', 'mean_atomic_radius'},
 {'range_Density', 'std_Density', 'wtd_std_Density'},
 {'gmean_ElectronAffinity', 'mean_ElectronAffinity

In [12]:
X_train_t = sel.transform(X_train)
X_test_t = sel.transform(X_test)

X_train_t.shape, X_test_t.shape

((16954, 31), (4239, 31))

In [13]:
X_train_t.head()

,gmean_atomic_mass,std_atomic_mass,gmean_fie,wtd_entropy_fie,wtd_range_fie,wtd_std_fie,gmean_atomic_radius,entropy_atomic_radius,wtd_range_atomic_radius,mean_Density,...,std_FusionHeat,mean_ThermalConductivity,gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_gmean_Valence,wtd_range_Valence,std_Valence
15511,18.464990,43.098264,918.814666,0.711383,592.635326,253.512834,110.098892,1.002546,89.255435,6977.029967,...,7.729959,54.060167,10.525894,0.693425,0.109671,37.925750,38.963777,2.112285,2.125000,1.247219
2892,52.075936,50.602625,808.929244,0.738144,652.408886,353.406612,110.717078,0.972142,34.555712,5035.809667,...,5.260175,137.675527,5.170344,0.140516,0.232107,56.402156,185.567013,2.242349,0.871650,0.471405
3650,60.708840,35.914688,663.564846,1.046750,695.457692,347.495347,155.077951,1.865338,22.800000,3861.061286,...,3.854102,99.860940,18.196006,1.144411,0.723486,61.524149,137.774946,2.102339,1.030769,0.451754
9028,72.746865,70.392496,838.584885,0.983628,596.564755,343.331641,131.729022,1.260175,43.130227,6501.357250,...,5.037900,106.581645,6.313081,0.270718,0.374856,47.775848,169.524558,2.000000,0.805257,0.000000
19291,80.031150,34.981246,696.141845,1.222394,345.123600,147.820700,168.727482,1.577281,48.848000,6075.000000,...,7.009422,55.600000,45.431663,1.445288,1.177546,26.836000,30.676375,3.567305,1.824000,1.019804


# Scaling

In [14]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_sc= min_max_scaler.fit_transform(X_train_t)

X_test_sc=min_max_scaler.fit_transform(X_test_t)

In [15]:
scaler = MinMaxScaler()


y_train_sc = scaler.fit_transform(y_train.values.reshape(-1, 1))


y_test_sc = scaler.transform(y_test.values.reshape(-1, 1))

# Baseline

In [16]:
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.dummy import DummyRegressor

In [17]:
from sklearn.metrics import (
    mean_absolute_error as mae,
    r2_score as r2,
    mean_absolute_percentage_error as mape,
    mean_squared_error as mse)

In [18]:
def report_metrics(model, X_train_sc, X_test_sc, y_train_sc, y_test_sc, label):
    print(f'Train MAE ({label}):', round(mae(y_train_sc, model.predict(X_train_sc)), 4))
    print(f'Test MAE ({label}) :', round(mae(y_test_sc, model.predict(X_test_sc)), 4), '\n')

    print(f'Train R^2 ({label}):', round(r2(y_train_sc, model.predict(X_train_sc)), 4))
    print(f'Test R^2 ({label}) :', round(r2(y_test_sc, model.predict(X_test_sc)), 4), '\n')

In [19]:
LR_baseline = LinearRegression().fit(X_train_sc, y_train_sc)
report_metrics(LR_baseline, X_train_sc, X_test_sc, y_train_sc, y_test_sc, 'LR_baseline')

Train MAE (LR_baseline): 0.0912
Test MAE (LR_baseline) : 0.0964 

Train R^2 (LR_baseline): 0.6118
Test R^2 (LR_baseline) : 0.5723 



# Cross-Validation

In [20]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import lightgbm as ltb

In [25]:
list_of_models = [        
        RandomForestRegressor(),
        XGBRegressor(),
        KNeighborsRegressor(),
        LGBMRegressor(),
        SVR()]
        
    

list_of_model_names = [type(x).__name__ for x in list_of_models]
cv_results = pd.DataFrame(
        data=0.0,
        index=list_of_model_names,
        columns=['fit_time', 'score_time','neg_mean_squared_error'])
    
    

    # обучение всех моделей из списка
for model in list_of_models:
        
        cv_result = cross_validate(
            estimator=model,
            X=X_train_sc,
            y=y_train_sc,
            scoring=['neg_mean_squared_error'],
            cv=30,
            n_jobs=-1)
        

        cv_results.loc[type(model).__name__] = list(map(np.mean, cv_result.values()))

print(cv_results)

                        fit_time  score_time  neg_mean_squared_error
RandomForestRegressor  71.786592    0.043830               -0.002814
XGBRegressor           12.280894    0.004793               -0.003030
KNeighborsRegressor     0.005051    0.105326               -0.003519
LGBMRegressor           0.808030    0.006320               -0.003386
SVR                     6.175440    0.393397               -0.006972


In [26]:
list_of_models = [        
        CatBoostRegressor()]
        
    

list_of_model_names = [type(x).__name__ for x in list_of_models]
cv_results = pd.DataFrame(
        data=0.0,
        index=list_of_model_names,
        columns=['fit_time', 'score_time','neg_mean_squared_error'])
    
    

    # обучение всех моделей из списка
for model in list_of_models:
        
        cv_result = cross_validate(
            estimator=model,
            X=X_train_sc,
            y=y_train_sc,
            scoring=['neg_mean_squared_error'],
            cv=5,
            n_jobs=-1)
        

        cv_results.loc[type(model).__name__] = list(map(np.mean, cv_result.values()))

print(cv_results)

                    fit_time  score_time  neg_mean_squared_error
CatBoostRegressor  20.175434    0.068045               -0.003088


# RandomForestRegressor

In [21]:
model_1 = RandomForestRegressor(n_estimators=100, random_state=42)
model_1.fit(X_train_sc, y_train_sc)

# Make predictions on the test set
y_pred_1 = model_1.predict(X_test_sc)

# Evaluate the model's performance
mse = mean_squared_error(y_test_sc, y_pred_1)
mae = mean_absolute_error(y_test_sc, y_pred_1)
r2 = r2_score(y_test_sc, y_pred_1)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_1)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

C:\Users\Misi\AppData\Local\Temp\ipykernel_16924\1833622144.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_1.fit(X_train_sc, y_train_sc)


Mean Squared Error (MSE): 0.007358041295099341
Mean Absolute Error (MAE): 0.0573509661566075
R-squared (R2): 0.7864984549625592
mape: 7.161263088296765


## tuning

In [41]:
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize


param_space = {
    'n_estimators': Integer(100, 1000),
    
    'max_depth': Integer(5, 50),
    'min_samples_split': Real(0.01, 0.5),
    'min_samples_leaf': Real(0.01, 0.5),
    
    'max_leaf_nodes': Integer(10, 100),
    
    'max_samples': Real(0.1, 1.0)
}

# Create Random Forest Regressor object
rf = RandomForestRegressor()

# Define Bayesian Search Cross Validation object with 20 iterations
bayes_cv = BayesSearchCV(
    rf,
    param_space,
    n_iter=50,
    cv=5,
    verbose=0,
    n_jobs=-1
)

# Fit the model with bayesian optimization search
bayes_cv.fit(X_train_sc, y_train_sc)

# Get the best hyperparameters
best_params = bayes_cv.best_params_

# Create a new Random Forest Regressor object with the best hyperparameters
best_rf = RandomForestRegressor(**best_params)

# Evaluate the best model using cross-validation
cv_scores = cross_val_score(best_rf, X_train_sc, y_train_sc, cv=5)

# Print the best hyperparameters and the cross-validation scores
print("Best hyperparameters: ", best_params)
print("Cross-validation scores: ", cv_scores)

C:\Users\Misi\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Misi\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Misi\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Misi\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed 

Best hyperparameters:  OrderedDict([('max_depth', 50), ('max_leaf_nodes', 100), ('max_samples', 1.0), ('min_samples_leaf', 0.01), ('min_samples_split', 0.01), ('n_estimators', 1000)])
Cross-validation scores:  [0.80062454 0.80986735 0.82407791 0.81707019 0.82100419]


In [44]:
best_params = {
   
    'max_depth': 50,
    
    'max_leaf_nodes': 100,
    'max_samples':  1,
    'min_samples_leaf': 0.01,
    'min_samples_split': 0.01,
    'n_estimators': 1000,
   
}

# Create a new Random Forest Regressor object with the best hyperparameters
rf = RandomForestRegressor(**best_params)

# Fit the model on the training data
rf.fit(X_train_sc, y_train_sc)

# Make predictions on the training data
y_pred_1_tun = rf.predict(X_test_sc)
mse = mean_squared_error(y_test_sc, y_pred_1_tun)
mae = mean_absolute_error(y_test_sc, y_pred_1_tun)
r2 = r2_score(y_test_sc, y_pred_1_tun)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_1_tun)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

C:\Users\Misi\AppData\Local\Temp\ipykernel_10116\3045707646.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train_sc, y_train_sc)


Mean Squared Error (MSE): 0.03452898690971699
Mean Absolute Error (MAE): 0.15735233049453415
R-squared (R2): -0.0018959880956492814
mape: 24.950317844231687


# XGBRegressor

In [22]:
model_2 = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model_2.fit(X_train_sc, y_train_sc)

# Make predictions on the test set
y_pred_2 = model_2.predict(X_test_sc)

# Evaluate the model's performance
mse = mean_squared_error(y_test_sc, y_pred_2)
mae = mean_absolute_error(y_test_sc, y_pred_2)
r2 = r2_score(y_test_sc, y_pred_2)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_2)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

Mean Squared Error (MSE): 0.007491251472332011
Mean Absolute Error (MAE): 0.05821806129273212
R-squared (R2): 0.7826332172568632
mape: 6.813304683419185


# Tuning xgboost

In [24]:
import xgboost as xgb
from skopt import gp_minimize

#### 200 iter, 10 cv

In [28]:
param_space = {
    'eta': Real(0.001, 1.0, 'log-uniform'),
    'gamma': Real(0.001, 1.0, 'log-uniform'),
    'max_depth': Integer(2, 15),
    'min_child_weight': Integer(1, 10),
    'max_delta_step': Real(0.001, 10.0, 'log-uniform'),
    'lambda': Real(0.001, 10.0, 'log-uniform'),
    'alpha': Real(0.001, 10.0, 'log-uniform'),
    'tree_method': (['auto', 'exact', 'approx', 'hist', 'gpu_hist']),
    'learning_rate': Real(0.001, 1.0, 'log-uniform'),    
    'max_leaves': Integer(2, 256)
}
model_10 = XGBRegressor(random_state=42)

# Define the search algorithm
search = BayesSearchCV(
    estimator=model_10,
    search_spaces=param_space,
    n_iter=200, # Number of iterations for optimization
    scoring='neg_mean_squared_error',
    cv=10, # Number of cross-validation folds
    n_jobs=-1, # Use all available CPU cores
    verbose=0
)

# Perform the search
search.fit(X_train_sc, y_train_sc)

# Print the best hyperparameters and the corresponding MSE score
print("Best hyperparameters:", search.best_params_)
print("Best MAPE score:", -search.best_score_)

Best hyperparameters: OrderedDict([('alpha', 0.001), ('eta', 1.0), ('gamma', 0.001), ('lambda', 0.001), ('learning_rate', 0.06652312842599188), ('max_delta_step', 2.3411714076378094), ('max_depth', 15), ('max_leaves', 256), ('min_child_weight', 10), ('tree_method', 'approx')])
Best MAPE score: 0.002756883841256313


In [33]:
model_2_tun = XGBRegressor(random_state=42, alpha=0.001, 
                            eta=1.0, gamma=0.001, reg_lambda= 0.001, 
                            learning_rate=0.06652312842599188, 
                            max_delta_step=2.3411714076378094, max_depth=15,
                            max_leaves=256, min_child_weight=10, tree_method='approx')
model_2_tun.fit(X_train_sc, y_train_sc)
y_pred_2_tun = model_2_tun.predict(X_test_sc)

# Evaluate the model's performance
mse = mean_squared_error(y_test_sc, y_pred_2_tun)
mae = mean_absolute_error(y_test_sc, y_pred_2_tun)
r2 = r2_score(y_test_sc, y_pred_2_tun)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_2_tun)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

Mean Squared Error (MSE): 0.007255391664319143
Mean Absolute Error (MAE): 0.05567444149938739
R-squared (R2): 0.789476945282217
mape: 6.426940435909983


#### 50 iter, 5 cv

In [30]:
param_space = {
    'eta': Real(0.001, 1.0, 'log-uniform'),
    'gamma': Real(0.001, 1.0, 'log-uniform'),
    'max_depth': Integer(2, 15),
    'min_child_weight': Integer(1, 10),
    'max_delta_step': Real(0.001, 10.0, 'log-uniform'),
    'lambda': Real(0.001, 10.0, 'log-uniform'),
    'alpha': Real(0.001, 10.0, 'log-uniform'),
    'tree_method': (['auto', 'exact', 'approx', 'hist', 'gpu_hist']),
    'learning_rate': Real(0.001, 1.0, 'log-uniform'),    
    'max_leaves': Integer(2, 256)
}
model_10 = XGBRegressor(random_state=42)

# Define the search algorithm
search = BayesSearchCV(
    estimator=model_10,
    search_spaces=param_space,
    n_iter=50, # Number of iterations for optimization
    scoring='neg_mean_squared_error',
    cv=5, # Number of cross-validation folds
    n_jobs=-1, # Use all available CPU cores
    verbose=0
)

# Perform the search
search.fit(X_train_sc, y_train_sc)

# Print the best hyperparameters and the corresponding MSE score
print("Best hyperparameters:", search.best_params_)
print("Best MAPE score:", -search.best_score_)

Best hyperparameters: OrderedDict([('alpha', 0.19821949513216439), ('eta', 1.0), ('gamma', 0.001), ('lambda', 10.0), ('learning_rate', 0.15117273769290032), ('max_delta_step', 0.06424252776599633), ('max_depth', 15), ('max_leaves', 256), ('min_child_weight', 2), ('tree_method', 'approx')])
Best MAPE score: 0.002917283221338184


In [36]:
model_2_tun_2 = XGBRegressor(random_state=42, alpha=0.19821949513216439, 
                            eta=1.0, gamma=0.001, reg_lambda= 10.0, 
                            learning_rate=0.15117273769290032, 
                            max_delta_step=0.06424252776599633, max_depth=15,
                            max_leaves=256, min_child_weight=2, tree_method='approx')
model_2_tun_2.fit(X_train_sc, y_train_sc)
y_pred_2_tun_2 = model_2_tun_2.predict(X_test_sc)

# Evaluate the model's performance
mse_2 = mean_squared_error(y_test_sc, y_pred_2_tun_2)
mae_2 = mean_absolute_error(y_test_sc, y_pred_2_tun_2)
r2_2 = r2_score(y_test_sc, y_pred_2_tun_2)
mape_2 = mean_absolute_percentage_error(y_test_sc, y_pred_2_tun_2)

print("Mean Squared Error (MSE):", mse_2)
print("Mean Absolute Error (MAE):", mae_2)
print("R-squared (R2):", r2_2)
print("mape:", mape_2)

Mean Squared Error (MSE): 0.007381215522244365
Mean Absolute Error (MAE): 0.057434208330081865
R-squared (R2): 0.7858260296387429
mape: 7.015006455659155


## Tuning Hyperopt

In [37]:


# Define the search space for hyperparameters
param_space = {
    'eta': hp.loguniform('eta', np.log(0.001), np.log(1.0)),
    'gamma': hp.loguniform('gamma', np.log(0.001), np.log(1.0)),
    'max_depth': hp.quniform('max_depth', 2, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'max_delta_step': hp.loguniform('max_delta_step', np.log(0.001), np.log(10.0)),
    'lambda': hp.loguniform('lambda', np.log(0.001), np.log(10.0)),
    'alpha': hp.loguniform('alpha', np.log(0.001), np.log(10.0)),
    'tree_method': hp.choice('tree_method', ['hist', 'exact']),  # Only CPU-based tree methods
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(1.0)),
}

# Define the objective function to minimize (negative mean squared error)
def objective(params):
    params['max_depth'] = int(params['max_depth'])  # Convert max_depth to an integer
    model = XGBRegressor(random_state=42, **params)
    mse_scores = -cross_val_score(model, X_train_sc, y_train_sc, scoring='neg_mean_squared_error', cv=5)
    return {'loss': mse_scores.mean(), 'status': STATUS_OK}

# Initialize Trials to keep track of results
trials = Trials()

# Define the number of iterations for optimization
num_iterations = 100

# Run the hyperparameter optimization with a progress bar
best = []
for _ in tqdm(range(num_iterations), desc='Hyperparameter Tuning', unit='iter'):
    best = fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=len(trials.trials) + 1,  # Incremental evaluations
        trials=trials,
        verbose=0
    )

# Get the best hyperparameters and corresponding MSE score
best_params = {param: value for param, value in best.items()}
best_mse = min([trial['result']['loss'] for trial in trials.trials])

print("Best hyperparameters:", best_params)
print("Best MSE score:", best_mse)

Hyperparameter Tuning: 100%|██████████| 100/100 [07:27<00:00,  4.48s/iter]

Best hyperparameters: {'alpha': 0.001037013736641145, 'eta': 0.00895847692220948, 'gamma': 0.0018267747767474528, 'lambda': 0.0016103032984844156, 'learning_rate': 0.08616973299988044, 'max_delta_step': 0.19918640112056257, 'max_depth': 11.0, 'min_child_weight': 3.0, 'tree_method': 1}
Best MSE score: 0.0029031008649062578


In [39]:
model_2_tun_3 = XGBRegressor(random_state=42, alpha=0.001037013736641145, 
                            eta=0.00895847692220948, gamma=0.0018267747767474528, reg_lambda= 0.0016103032984844156, 
                            learning_rate=0.08616973299988044, 
                            max_delta_step=0.19918640112056257, max_depth=11,
                             min_child_weight=3.0, tree_method='exact')
model_2_tun_3.fit(X_train_sc, y_train_sc)
y_pred_2_tun_3 = model_2_tun_3.predict(X_test_sc)

# Evaluate the model's performance
mse_3 = mean_squared_error(y_test_sc, y_pred_2_tun_3)
mae_3 = mean_absolute_error(y_test_sc, y_pred_2_tun_3)
r2_3 = r2_score(y_test_sc, y_pred_2_tun_3)
mape_3 = mean_absolute_percentage_error(y_test_sc, y_pred_2_tun_3)

print("Mean Squared Error (MSE):", mse_3)
print("Mean Absolute Error (MAE):", mae_3)
print("R-squared (R2):", r2_3)
print("mape:", mape_3)

Mean Squared Error (MSE): 0.007229605809600556
Mean Absolute Error (MAE): 0.05722120320278166
R-squared (R2): 0.7902251498113484
mape: 7.501843401250104


In [24]:
param_space = {
    'eta': hp.loguniform('eta', np.log(0.001), np.log(1.0)),
    'gamma': hp.loguniform('gamma', np.log(0.001), np.log(1.0)),
    'max_depth': hp.quniform('max_depth', 2, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'max_delta_step': hp.loguniform('max_delta_step', np.log(0.001), np.log(10.0)),
    'lambda': hp.loguniform('lambda', np.log(0.001), np.log(10.0)),
    'alpha': hp.loguniform('alpha', np.log(0.001), np.log(10.0)),
    'tree_method': hp.choice('tree_method', ['hist', 'exact']),  # Only CPU-based tree methods
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(1.0)),
}

# Define the objective function to minimize (negative mean squared error)
def objective(params):
    params['max_depth'] = int(params['max_depth'])  # Convert max_depth to an integer
    model = XGBRegressor(random_state=42, **params)
    mse_scores = -cross_val_score(model, X_train_sc, y_train_sc, scoring='neg_mean_squared_error', cv=5)
    return {'loss': mse_scores.mean(), 'status': STATUS_OK}

# Initialize Trials to keep track of results
trials = Trials()

# Define the number of iterations for optimization
num_iterations = 50

# Run the hyperparameter optimization with a progress bar
best = []
for _ in tqdm(range(num_iterations), desc='Hyperparameter Tuning', unit='iter'):
    best = fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=len(trials.trials) + 1,  # Incremental evaluations
        trials=trials,
        verbose=0
    )

# Get the best hyperparameters and corresponding MSE score
best_params = {param: value for param, value in best.items()}
best_mse = min([trial['result']['loss'] for trial in trials.trials])

print("Best hyperparameters:", best_params)
print("Best MSE score:", best_mse)

Hyperparameter Tuning: 100%|██████████| 50/50 [02:43<00:00,  3.26s/iter]

Best hyperparameters: {'alpha': 0.001523252049840121, 'eta': 0.012375966057082168, 'gamma': 0.0010980102402781073, 'lambda': 8.921307217849604, 'learning_rate': 0.1610386949328752, 'max_delta_step': 0.5214327295339104, 'max_depth': 12.0, 'min_child_weight': 3.0, 'tree_method': 1}
Best MSE score: 0.0029198563727147963


In [26]:
model_2_tun_4 = XGBRegressor(random_state=42, alpha=0.001523252049840121, 
                            eta=0.012375966057082168, gamma=0.0010980102402781073, reg_lambda= 8.921307217849604, 
                            learning_rate=0.1610386949328752, 
                            max_delta_step=0.5214327295339104, max_depth=12,
                             min_child_weight=3.0, tree_method='exact')
model_2_tun_4.fit(X_train_sc, y_train_sc)
y_pred_2_tun_4 = model_2_tun_4.predict(X_test_sc)

# Evaluate the model's performance
mse_3 = mean_squared_error(y_test_sc, y_pred_2_tun_4)
mae_3 = mean_absolute_error(y_test_sc, y_pred_2_tun_4)
r2_3 = r2_score(y_test_sc, y_pred_2_tun_4)
mape_3 = mean_absolute_percentage_error(y_test_sc, y_pred_2_tun_4)

print("Mean Squared Error (MSE):", mse_3)
print("Mean Absolute Error (MAE):", mae_3)
print("R-squared (R2):", r2_3)
print("mape:", mape_3)

Mean Squared Error (MSE): 0.00864258968887564
Mean Absolute Error (MAE): 0.06212881431295912
R-squared (R2): 0.7492258907369055
mape: 9.003596118770613


# CatBoostRegressor

In [23]:
model_3 = CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, random_seed=42)


model_3.fit(X_train_sc, y_train_sc, verbose=50)


y_pred_3 = model_3.predict(X_test_sc)

mse = mean_squared_error(y_test_sc, y_pred_3)
mae = mean_absolute_error(y_test_sc, y_pred_3)
r2 = r2_score(y_test_sc, y_pred_3)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_3)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

0:	learn: 0.1725882	total: 143ms	remaining: 28.5s
50:	learn: 0.0710694	total: 279ms	remaining: 817ms
100:	learn: 0.0635301	total: 415ms	remaining: 407ms
150:	learn: 0.0587819	total: 553ms	remaining: 180ms
199:	learn: 0.0556116	total: 691ms	remaining: 0us
Mean Squared Error (MSE): 0.006784431637551927
Mean Absolute Error (MAE): 0.056308466943344224
R-squared (R2): 0.803142361027117
mape: 6.329450476676215


In [34]:
from hyperopt import hp, fmin, tpe, Trials,  space_eval
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'depth': hp.choice('depth', [2, 4, 6, 8, 10]),
    'iterations': hp.choice('iterations', [50, 100, 200, 400, 800]),
    'model_size_reg': hp.uniform('model_size_reg', 0, 1),
    'boosting_type': hp.choice('boosting_type', ['Ordered', 'Plain']),
    'reg_lambda': hp.loguniform('reg_lambda', -9, 2),
    'max_bin': hp.choice('max_bin', [8, 16, 32, 64, 128]),
    'min_child_samples': hp.choice('min_child_samples', [2, 5, 10, 15, 20])
}

# define the objective function to minimize (negative mean squared error)
def objective(params):
    model = CatBoostRegressor(
        learning_rate=params['learning_rate'],
        depth=params['depth'],
        iterations=params['iterations'],
        model_size_reg=params['model_size_reg'],
        boosting_type=params['boosting_type'],
        reg_lambda=params['reg_lambda'],
        max_bin=params['max_bin'],
        min_child_samples=params['min_child_samples'],
        silent=True,
        random_state=42
    )
    scores = -cross_val_score(model, X_train_sc, y_train_sc, cv=10, scoring='neg_mean_squared_error')
    score = np.mean(scores)
    return score

# set up the hyperparameter tuning using the Tree-structured Parzen Estimator (TPE) algorithm
trials = Trials()
random_state= np.random.default_rng(seed=42)
#random_state = np.random.RandomState(seed=42)
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials, 
            rstate=random_state) 
            
# print the best hyperparameters and the corresponding mean cross-validation score

best_params = space_eval(space, best)
model_group4 = CatBoostRegressor(**best_params, silent=True, random_state=42)
model_group4.fit(X_train_sc, y_train_sc)
y_pred = model_group4.predict(X_test_sc)


print("Best hyperparameters:", best_params)
print('r2 :', r2(y_test_sc, y_pred))
print("Best MSE score:",  mse(y_test_sc, y_pred))
print('mae:', mae(y_test_sc, y_pred))
print('mape:', mape(y_test_sc, y_pred))

100%|██████████| 100/100 [44:34<00:00, 26.75s/trial, best loss: 0.002724585543236481]  
Best hyperparameters: {'boosting_type': 'Plain', 'depth': 8, 'iterations': 800, 'learning_rate': 0.0793431083796097, 'max_bin': 128, 'min_child_samples': 10, 'model_size_reg': 0.40060028201354186, 'reg_lambda': 0.04204952184594683}


TypeError: 'numpy.float64' object is not callable

In [35]:
best_params = {'boosting_type': 'Plain', 'depth': 8,
               'iterations': 800, 'learning_rate': 0.0793431083796097,
               'max_bin': 128, 'min_child_samples': 10, 
               'model_size_reg': 0.40060028201354186, 
               'reg_lambda': 0.04204952184594683}

model_3_tun = CatBoostRegressor(**best_params, silent=True, random_state=42)
model_3_tun.fit(X_train_sc, y_train_sc)
y_pred_3_tun = model_3_tun.predict(X_test_sc)

mse = mean_squared_error(y_test_sc, y_pred_3_tun)
mae = mean_absolute_error(y_test_sc, y_pred_3_tun)
r2 = r2_score(y_test_sc, y_pred_3_tun)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_3_tun)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

Mean Squared Error (MSE): 0.00656420085916905
Mean Absolute Error (MAE): 0.05379922932791652
R-squared (R2): 0.8095325958143094
mape: 4.773120985754883


In [40]:
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'depth': hp.choice('depth', [2, 4, 6, 8, 10]),
    'iterations': hp.choice('iterations', [50, 100, 200, 400, 800]),
    'model_size_reg': hp.uniform('model_size_reg', 0, 1),
    'boosting_type': hp.choice('boosting_type', ['Ordered', 'Plain']),
    'reg_lambda': hp.loguniform('reg_lambda', -9, 2),
    'max_bin': hp.choice('max_bin', [8, 16, 32, 64, 128]),
    'min_child_samples': hp.choice('min_child_samples', [2, 5, 10, 15, 20])
}

# define the objective function to minimize (negative mean squared error)
def objective(params):
    model = CatBoostRegressor(
        learning_rate=params['learning_rate'],
        depth=params['depth'],
        iterations=params['iterations'],
        model_size_reg=params['model_size_reg'],
        boosting_type=params['boosting_type'],
        reg_lambda=params['reg_lambda'],
        max_bin=params['max_bin'],
        min_child_samples=params['min_child_samples'],
        silent=True,
        random_state=42
    )
    scores = -cross_val_score(model, X_train_sc, y_train_sc, cv=5, scoring='neg_mean_squared_error')
    score = np.mean(scores)
    return score

# set up the hyperparameter tuning using the Tree-structured Parzen Estimator (TPE) algorithm
trials = Trials()
random_state= np.random.default_rng(seed=42)
#random_state = np.random.RandomState(seed=42)
best = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=trials, 
            rstate=random_state) 
            
# print the best hyperparameters and the corresponding mean cross-validation score

best_params = space_eval(space, best)
model_3_tun = CatBoostRegressor(**best_params, silent=True, random_state=42)
model_3_tun.fit(X_train_sc, y_train_sc)
y_pred_3_tun_3 = model_3_tun.predict(X_test_sc)


mse = mean_squared_error(y_test_sc, y_pred_3_tun_3)
mae = mean_absolute_error(y_test_sc, y_pred_3_tun_3)
r2 = r2_score(y_test_sc, y_pred_3_tun_3)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_3_tun_3)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)


100%|██████████| 50/50 [17:07<00:00, 20.54s/trial, best loss: 0.0028485621867172578]
Mean Squared Error (MSE): 0.006533769166704837
Mean Absolute Error (MAE): 0.05390477770999708
R-squared (R2): 0.8104156043621876
mape: 5.634064005321226


In [27]:
# Define the hyperparameter search space
param_space = {
    'learning_rate': Real(0.01, 0.3, 'log-uniform'),
    'depth': Integer(3, 10),
    'l2_leaf_reg': Real(1.0, 10.0, 'log-uniform'),
    'border_count': Integer(1, 100),
    'bagging_temperature': Real(0.1, 0.9),
    'min_data_in_leaf': Integer(5, 50),
    'grow_policy': Categorical(['SymmetricTree', 'Depthwise', 'Lossguide']),
}

# Create the CatBoostRegressor model
model = CatBoostRegressor(random_state=42)

# Define the search algorithm
search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=0
)

# Perform the search
search.fit(X_train_sc, y_train_sc)

# Print the best hyperparameters and the corresponding MSE score
print("Best hyperparameters:", search.best_params_)
print("Best MSE score:", -search.best_score_)


0:	learn: 0.1779072	total: 141ms	remaining: 2m 21s
1:	learn: 0.1708408	total: 149ms	remaining: 1m 14s
2:	learn: 0.1645382	total: 156ms	remaining: 52s
3:	learn: 0.1582538	total: 165ms	remaining: 41s
4:	learn: 0.1523706	total: 174ms	remaining: 34.6s
5:	learn: 0.1469632	total: 182ms	remaining: 30.1s
6:	learn: 0.1420143	total: 189ms	remaining: 26.8s
7:	learn: 0.1373849	total: 197ms	remaining: 24.5s
8:	learn: 0.1329347	total: 204ms	remaining: 22.5s
9:	learn: 0.1283746	total: 213ms	remaining: 21.1s
10:	learn: 0.1243651	total: 221ms	remaining: 19.9s
11:	learn: 0.1207990	total: 229ms	remaining: 18.8s
12:	learn: 0.1170900	total: 237ms	remaining: 18s
13:	learn: 0.1138337	total: 245ms	remaining: 17.2s
14:	learn: 0.1106100	total: 253ms	remaining: 16.6s
15:	learn: 0.1078839	total: 261ms	remaining: 16.1s
16:	learn: 0.1049781	total: 269ms	remaining: 15.6s
17:	learn: 0.1020643	total: 278ms	remaining: 15.2s
18:	learn: 0.0993969	total: 286ms	remaining: 14.8s
19:	learn: 0.0969906	total: 295ms	remaining: 

189:	learn: 0.0445426	total: 1.62s	remaining: 6.92s
190:	learn: 0.0445060	total: 1.63s	remaining: 6.9s
191:	learn: 0.0444936	total: 1.64s	remaining: 6.88s
192:	learn: 0.0444238	total: 1.64s	remaining: 6.87s
193:	learn: 0.0443440	total: 1.65s	remaining: 6.86s
194:	learn: 0.0443138	total: 1.66s	remaining: 6.84s
195:	learn: 0.0442559	total: 1.66s	remaining: 6.82s
196:	learn: 0.0442101	total: 1.67s	remaining: 6.8s
197:	learn: 0.0440874	total: 1.68s	remaining: 6.79s
198:	learn: 0.0440374	total: 1.68s	remaining: 6.77s
199:	learn: 0.0439465	total: 1.69s	remaining: 6.75s
200:	learn: 0.0438748	total: 1.7s	remaining: 6.74s
201:	learn: 0.0437747	total: 1.7s	remaining: 6.73s
202:	learn: 0.0436789	total: 1.71s	remaining: 6.72s
203:	learn: 0.0436291	total: 1.72s	remaining: 6.71s
204:	learn: 0.0435039	total: 1.73s	remaining: 6.7s
205:	learn: 0.0434605	total: 1.74s	remaining: 6.69s
206:	learn: 0.0433822	total: 1.74s	remaining: 6.67s
207:	learn: 0.0433225	total: 1.75s	remaining: 6.66s
208:	learn: 0.043

351:	learn: 0.0367657	total: 2.68s	remaining: 4.93s
352:	learn: 0.0367120	total: 2.69s	remaining: 4.92s
353:	learn: 0.0366614	total: 2.69s	remaining: 4.92s
354:	learn: 0.0366285	total: 2.7s	remaining: 4.91s
355:	learn: 0.0366035	total: 2.71s	remaining: 4.89s
356:	learn: 0.0365672	total: 2.71s	remaining: 4.88s
357:	learn: 0.0365308	total: 2.72s	remaining: 4.88s
358:	learn: 0.0365150	total: 2.73s	remaining: 4.87s
359:	learn: 0.0364791	total: 2.73s	remaining: 4.85s
360:	learn: 0.0364622	total: 2.74s	remaining: 4.84s
361:	learn: 0.0364260	total: 2.74s	remaining: 4.84s
362:	learn: 0.0364002	total: 2.75s	remaining: 4.83s
363:	learn: 0.0363888	total: 2.76s	remaining: 4.82s
364:	learn: 0.0363614	total: 2.76s	remaining: 4.8s
365:	learn: 0.0363360	total: 2.77s	remaining: 4.79s
366:	learn: 0.0363097	total: 2.77s	remaining: 4.78s
367:	learn: 0.0362676	total: 2.78s	remaining: 4.77s
368:	learn: 0.0362546	total: 2.78s	remaining: 4.76s
369:	learn: 0.0362471	total: 2.79s	remaining: 4.75s
370:	learn: 0.

522:	learn: 0.0328514	total: 3.71s	remaining: 3.38s
523:	learn: 0.0328271	total: 3.72s	remaining: 3.38s
524:	learn: 0.0328152	total: 3.73s	remaining: 3.37s
525:	learn: 0.0327826	total: 3.73s	remaining: 3.36s
526:	learn: 0.0327737	total: 3.74s	remaining: 3.35s
527:	learn: 0.0327498	total: 3.75s	remaining: 3.35s
528:	learn: 0.0327248	total: 3.75s	remaining: 3.34s
529:	learn: 0.0327175	total: 3.76s	remaining: 3.33s
530:	learn: 0.0327004	total: 3.76s	remaining: 3.32s
531:	learn: 0.0326905	total: 3.77s	remaining: 3.32s
532:	learn: 0.0326736	total: 3.77s	remaining: 3.31s
533:	learn: 0.0326421	total: 3.78s	remaining: 3.3s
534:	learn: 0.0326238	total: 3.79s	remaining: 3.29s
535:	learn: 0.0326130	total: 3.79s	remaining: 3.28s
536:	learn: 0.0326088	total: 3.8s	remaining: 3.28s
537:	learn: 0.0325935	total: 3.81s	remaining: 3.27s
538:	learn: 0.0325780	total: 3.81s	remaining: 3.26s
539:	learn: 0.0325492	total: 3.82s	remaining: 3.25s
540:	learn: 0.0325308	total: 3.82s	remaining: 3.24s
541:	learn: 0.

695:	learn: 0.0304542	total: 4.74s	remaining: 2.07s
696:	learn: 0.0304477	total: 4.75s	remaining: 2.06s
697:	learn: 0.0304447	total: 4.76s	remaining: 2.06s
698:	learn: 0.0304398	total: 4.76s	remaining: 2.05s
699:	learn: 0.0304356	total: 4.77s	remaining: 2.04s
700:	learn: 0.0304152	total: 4.77s	remaining: 2.04s
701:	learn: 0.0304093	total: 4.78s	remaining: 2.03s
702:	learn: 0.0304071	total: 4.78s	remaining: 2.02s
703:	learn: 0.0304046	total: 4.79s	remaining: 2.01s
704:	learn: 0.0303881	total: 4.79s	remaining: 2.01s
705:	learn: 0.0303816	total: 4.8s	remaining: 2s
706:	learn: 0.0303765	total: 4.81s	remaining: 1.99s
707:	learn: 0.0303613	total: 4.81s	remaining: 1.98s
708:	learn: 0.0303443	total: 4.82s	remaining: 1.98s
709:	learn: 0.0303394	total: 4.82s	remaining: 1.97s
710:	learn: 0.0303200	total: 4.83s	remaining: 1.96s
711:	learn: 0.0303014	total: 4.84s	remaining: 1.96s
712:	learn: 0.0302947	total: 4.84s	remaining: 1.95s
713:	learn: 0.0302873	total: 4.85s	remaining: 1.94s
714:	learn: 0.03

874:	learn: 0.0289452	total: 5.79s	remaining: 827ms
875:	learn: 0.0289396	total: 5.79s	remaining: 820ms
876:	learn: 0.0289382	total: 5.8s	remaining: 813ms
877:	learn: 0.0289258	total: 5.81s	remaining: 807ms
878:	learn: 0.0289223	total: 5.81s	remaining: 800ms
879:	learn: 0.0289178	total: 5.82s	remaining: 793ms
880:	learn: 0.0289117	total: 5.82s	remaining: 787ms
881:	learn: 0.0289026	total: 5.83s	remaining: 780ms
882:	learn: 0.0289012	total: 5.83s	remaining: 773ms
883:	learn: 0.0288914	total: 5.84s	remaining: 766ms
884:	learn: 0.0288806	total: 5.84s	remaining: 760ms
885:	learn: 0.0288792	total: 5.85s	remaining: 753ms
886:	learn: 0.0288732	total: 5.86s	remaining: 746ms
887:	learn: 0.0288647	total: 5.86s	remaining: 739ms
888:	learn: 0.0288486	total: 5.87s	remaining: 733ms
889:	learn: 0.0288394	total: 5.87s	remaining: 726ms
890:	learn: 0.0288346	total: 5.88s	remaining: 719ms
891:	learn: 0.0288260	total: 5.89s	remaining: 713ms
892:	learn: 0.0288173	total: 5.89s	remaining: 706ms
893:	learn: 0

In [28]:
best_hyperparameters = {
    'bagging_temperature': 0.39821776354008,
    'border_count':50,
    'depth': 8,
    'grow_policy': 'Depthwise',
    'l2_leaf_reg': 1.0,
    'learning_rate': 0.05254324533602761,
    'min_data_in_leaf': 12,
}


model_3_tun_2 = CatBoostRegressor(
    bagging_temperature=best_hyperparameters['bagging_temperature'],
    border_count=best_hyperparameters['border_count'],
    depth=best_hyperparameters['depth'],
    grow_policy=best_hyperparameters['grow_policy'],
    l2_leaf_reg=best_hyperparameters['l2_leaf_reg'],
    learning_rate=best_hyperparameters['learning_rate'],
    min_data_in_leaf=best_hyperparameters['min_data_in_leaf'],
    random_state=42  
)


model_3_tun_2.fit(X_train_sc, y_train_sc)


y_pred_3_tun_2 = model_3_tun_2.predict(X_test_sc)


mse = mean_squared_error(y_test_sc, y_pred_3_tun_2)
mae = mean_absolute_error(y_test_sc, y_pred_3_tun_2)
r2 = r2_score(y_test_sc, y_pred_3_tun_2)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_3_tun_2)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)


0:	learn: 0.1779072	total: 8.06ms	remaining: 8.06s
1:	learn: 0.1708408	total: 17.2ms	remaining: 8.59s
2:	learn: 0.1645382	total: 26.2ms	remaining: 8.72s
3:	learn: 0.1582538	total: 36.1ms	remaining: 8.98s
4:	learn: 0.1523706	total: 45.2ms	remaining: 8.99s
5:	learn: 0.1469632	total: 52.7ms	remaining: 8.73s
6:	learn: 0.1420143	total: 60ms	remaining: 8.5s
7:	learn: 0.1373849	total: 69.2ms	remaining: 8.58s
8:	learn: 0.1329347	total: 75.4ms	remaining: 8.3s
9:	learn: 0.1283746	total: 84.9ms	remaining: 8.41s
10:	learn: 0.1243651	total: 92.7ms	remaining: 8.34s
11:	learn: 0.1207990	total: 100ms	remaining: 8.27s
12:	learn: 0.1170900	total: 108ms	remaining: 8.21s
13:	learn: 0.1138337	total: 116ms	remaining: 8.17s
14:	learn: 0.1106100	total: 124ms	remaining: 8.13s
15:	learn: 0.1078839	total: 132ms	remaining: 8.12s
16:	learn: 0.1049781	total: 140ms	remaining: 8.07s
17:	learn: 0.1020643	total: 148ms	remaining: 8.07s
18:	learn: 0.0993969	total: 156ms	remaining: 8.03s
19:	learn: 0.0969906	total: 164ms	

184:	learn: 0.0449517	total: 1.44s	remaining: 6.34s
185:	learn: 0.0448442	total: 1.45s	remaining: 6.33s
186:	learn: 0.0447605	total: 1.46s	remaining: 6.34s
187:	learn: 0.0446732	total: 1.46s	remaining: 6.32s
188:	learn: 0.0446112	total: 1.47s	remaining: 6.31s
189:	learn: 0.0445426	total: 1.48s	remaining: 6.3s
190:	learn: 0.0445060	total: 1.48s	remaining: 6.29s
191:	learn: 0.0444936	total: 1.49s	remaining: 6.26s
192:	learn: 0.0444238	total: 1.5s	remaining: 6.25s
193:	learn: 0.0443440	total: 1.5s	remaining: 6.25s
194:	learn: 0.0443138	total: 1.51s	remaining: 6.23s
195:	learn: 0.0442559	total: 1.51s	remaining: 6.21s
196:	learn: 0.0442101	total: 1.52s	remaining: 6.2s
197:	learn: 0.0440874	total: 1.53s	remaining: 6.19s
198:	learn: 0.0440374	total: 1.53s	remaining: 6.17s
199:	learn: 0.0439465	total: 1.54s	remaining: 6.16s
200:	learn: 0.0438748	total: 1.55s	remaining: 6.15s
201:	learn: 0.0437747	total: 1.55s	remaining: 6.14s
202:	learn: 0.0436789	total: 1.56s	remaining: 6.13s
203:	learn: 0.04

346:	learn: 0.0369595	total: 2.46s	remaining: 4.64s
347:	learn: 0.0369355	total: 2.47s	remaining: 4.63s
348:	learn: 0.0369254	total: 2.48s	remaining: 4.63s
349:	learn: 0.0368846	total: 2.48s	remaining: 4.62s
350:	learn: 0.0368224	total: 2.49s	remaining: 4.61s
351:	learn: 0.0367657	total: 2.5s	remaining: 4.6s
352:	learn: 0.0367120	total: 2.5s	remaining: 4.59s
353:	learn: 0.0366614	total: 2.51s	remaining: 4.58s
354:	learn: 0.0366285	total: 2.52s	remaining: 4.57s
355:	learn: 0.0366035	total: 2.52s	remaining: 4.56s
356:	learn: 0.0365672	total: 2.53s	remaining: 4.55s
357:	learn: 0.0365308	total: 2.53s	remaining: 4.54s
358:	learn: 0.0365150	total: 2.54s	remaining: 4.54s
359:	learn: 0.0364791	total: 2.55s	remaining: 4.53s
360:	learn: 0.0364622	total: 2.55s	remaining: 4.52s
361:	learn: 0.0364260	total: 2.56s	remaining: 4.51s
362:	learn: 0.0364002	total: 2.56s	remaining: 4.5s
363:	learn: 0.0363888	total: 2.57s	remaining: 4.49s
364:	learn: 0.0363614	total: 2.58s	remaining: 4.48s
365:	learn: 0.03

528:	learn: 0.0327248	total: 3.5s	remaining: 3.12s
529:	learn: 0.0327175	total: 3.51s	remaining: 3.11s
530:	learn: 0.0327004	total: 3.51s	remaining: 3.1s
531:	learn: 0.0326905	total: 3.52s	remaining: 3.1s
532:	learn: 0.0326736	total: 3.53s	remaining: 3.09s
533:	learn: 0.0326421	total: 3.53s	remaining: 3.08s
534:	learn: 0.0326238	total: 3.54s	remaining: 3.08s
535:	learn: 0.0326130	total: 3.54s	remaining: 3.07s
536:	learn: 0.0326088	total: 3.55s	remaining: 3.06s
537:	learn: 0.0325935	total: 3.56s	remaining: 3.05s
538:	learn: 0.0325780	total: 3.56s	remaining: 3.05s
539:	learn: 0.0325492	total: 3.57s	remaining: 3.04s
540:	learn: 0.0325308	total: 3.57s	remaining: 3.03s
541:	learn: 0.0325098	total: 3.58s	remaining: 3.02s
542:	learn: 0.0325063	total: 3.58s	remaining: 3.02s
543:	learn: 0.0325038	total: 3.59s	remaining: 3.01s
544:	learn: 0.0324910	total: 3.59s	remaining: 3s
545:	learn: 0.0324847	total: 3.6s	remaining: 2.99s
546:	learn: 0.0324658	total: 3.6s	remaining: 2.98s
547:	learn: 0.032461

713:	learn: 0.0302873	total: 4.54s	remaining: 1.82s
714:	learn: 0.0302813	total: 4.54s	remaining: 1.81s
715:	learn: 0.0302784	total: 4.55s	remaining: 1.8s
716:	learn: 0.0302764	total: 4.55s	remaining: 1.8s
717:	learn: 0.0302658	total: 4.56s	remaining: 1.79s
718:	learn: 0.0302574	total: 4.56s	remaining: 1.78s
719:	learn: 0.0302493	total: 4.57s	remaining: 1.78s
720:	learn: 0.0302442	total: 4.57s	remaining: 1.77s
721:	learn: 0.0302294	total: 4.58s	remaining: 1.76s
722:	learn: 0.0302100	total: 4.59s	remaining: 1.76s
723:	learn: 0.0301913	total: 4.59s	remaining: 1.75s
724:	learn: 0.0301886	total: 4.6s	remaining: 1.74s
725:	learn: 0.0301757	total: 4.61s	remaining: 1.74s
726:	learn: 0.0301609	total: 4.61s	remaining: 1.73s
727:	learn: 0.0301543	total: 4.62s	remaining: 1.73s
728:	learn: 0.0301449	total: 4.62s	remaining: 1.72s
729:	learn: 0.0301318	total: 4.63s	remaining: 1.71s
730:	learn: 0.0301104	total: 4.63s	remaining: 1.71s
731:	learn: 0.0301091	total: 4.64s	remaining: 1.7s
732:	learn: 0.03

903:	learn: 0.0287657	total: 5.58s	remaining: 593ms
904:	learn: 0.0287516	total: 5.59s	remaining: 587ms
905:	learn: 0.0287483	total: 5.59s	remaining: 580ms
906:	learn: 0.0287432	total: 5.6s	remaining: 574ms
907:	learn: 0.0287410	total: 5.6s	remaining: 568ms
908:	learn: 0.0287336	total: 5.61s	remaining: 561ms
909:	learn: 0.0287293	total: 5.61s	remaining: 555ms
910:	learn: 0.0287229	total: 5.62s	remaining: 549ms
911:	learn: 0.0287181	total: 5.62s	remaining: 543ms
912:	learn: 0.0287171	total: 5.63s	remaining: 536ms
913:	learn: 0.0287135	total: 5.63s	remaining: 530ms
914:	learn: 0.0287015	total: 5.64s	remaining: 524ms
915:	learn: 0.0286967	total: 5.64s	remaining: 518ms
916:	learn: 0.0286910	total: 5.65s	remaining: 512ms
917:	learn: 0.0286777	total: 5.66s	remaining: 505ms
918:	learn: 0.0286764	total: 5.66s	remaining: 499ms
919:	learn: 0.0286680	total: 5.67s	remaining: 493ms
920:	learn: 0.0286454	total: 5.68s	remaining: 487ms
921:	learn: 0.0286420	total: 5.68s	remaining: 481ms
922:	learn: 0.

In [29]:
# Define the hyperparameter search space
param_space = {
    'learning_rate': Real(0.01, 0.3, 'log-uniform'),
    'depth': Integer(3, 10),
    'l2_leaf_reg': Real(1.0, 10.0, 'log-uniform'),
    'border_count': Integer(1, 100),
    'bagging_temperature': Real(0.1, 0.9),
    'min_data_in_leaf': Integer(5, 50),
    'grow_policy': Categorical(['SymmetricTree', 'Depthwise', 'Lossguide']),
}

# Create the CatBoostRegressor model
model = CatBoostRegressor(random_state=42)

# Define the search algorithm
search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    n_iter=200,
    scoring='neg_mean_squared_error',
    cv=10,
    n_jobs=-1,
    verbose=0
)

# Perform the search
search.fit(X_train_sc, y_train_sc)

# Print the best hyperparameters and the corresponding MSE score
print("Best hyperparameters:", search.best_params_)
print("Best MSE score:", -search.best_score_)


0:	learn: 0.1783573	total: 13.6ms	remaining: 13.6s
1:	learn: 0.1715346	total: 28.5ms	remaining: 14.2s
2:	learn: 0.1652404	total: 40.6ms	remaining: 13.5s
3:	learn: 0.1590798	total: 56.5ms	remaining: 14.1s
4:	learn: 0.1534345	total: 73.9ms	remaining: 14.7s
5:	learn: 0.1479502	total: 88ms	remaining: 14.6s
6:	learn: 0.1428466	total: 102ms	remaining: 14.5s
7:	learn: 0.1381261	total: 117ms	remaining: 14.5s
8:	learn: 0.1336002	total: 130ms	remaining: 14.3s
9:	learn: 0.1291443	total: 143ms	remaining: 14.2s
10:	learn: 0.1250764	total: 157ms	remaining: 14.1s
11:	learn: 0.1212791	total: 172ms	remaining: 14.1s
12:	learn: 0.1176018	total: 186ms	remaining: 14.2s
13:	learn: 0.1140326	total: 202ms	remaining: 14.2s
14:	learn: 0.1107658	total: 216ms	remaining: 14.2s
15:	learn: 0.1078463	total: 231ms	remaining: 14.2s
16:	learn: 0.1049178	total: 246ms	remaining: 14.2s
17:	learn: 0.1022863	total: 261ms	remaining: 14.2s
18:	learn: 0.0997259	total: 273ms	remaining: 14.1s
19:	learn: 0.0970918	total: 286ms	rem

165:	learn: 0.0428818	total: 2.18s	remaining: 11s
166:	learn: 0.0428054	total: 2.2s	remaining: 11s
167:	learn: 0.0427552	total: 2.21s	remaining: 10.9s
168:	learn: 0.0426847	total: 2.22s	remaining: 10.9s
169:	learn: 0.0426420	total: 2.23s	remaining: 10.9s
170:	learn: 0.0425699	total: 2.24s	remaining: 10.9s
171:	learn: 0.0424999	total: 2.25s	remaining: 10.8s
172:	learn: 0.0424185	total: 2.26s	remaining: 10.8s
173:	learn: 0.0423666	total: 2.27s	remaining: 10.8s
174:	learn: 0.0423117	total: 2.28s	remaining: 10.8s
175:	learn: 0.0422517	total: 2.29s	remaining: 10.7s
176:	learn: 0.0421646	total: 2.31s	remaining: 10.7s
177:	learn: 0.0421279	total: 2.32s	remaining: 10.7s
178:	learn: 0.0420786	total: 2.33s	remaining: 10.7s
179:	learn: 0.0419883	total: 2.34s	remaining: 10.7s
180:	learn: 0.0419288	total: 2.35s	remaining: 10.6s
181:	learn: 0.0418479	total: 2.36s	remaining: 10.6s
182:	learn: 0.0417906	total: 2.37s	remaining: 10.6s
183:	learn: 0.0417117	total: 2.38s	remaining: 10.6s
184:	learn: 0.041

334:	learn: 0.0349336	total: 3.66s	remaining: 7.26s
335:	learn: 0.0348916	total: 3.67s	remaining: 7.25s
336:	learn: 0.0348839	total: 3.67s	remaining: 7.23s
337:	learn: 0.0348502	total: 3.68s	remaining: 7.21s
338:	learn: 0.0348100	total: 3.69s	remaining: 7.2s
339:	learn: 0.0347658	total: 3.7s	remaining: 7.19s
340:	learn: 0.0347328	total: 3.71s	remaining: 7.17s
341:	learn: 0.0347056	total: 3.72s	remaining: 7.15s
342:	learn: 0.0346749	total: 3.73s	remaining: 7.14s
343:	learn: 0.0346455	total: 3.73s	remaining: 7.12s
344:	learn: 0.0346217	total: 3.74s	remaining: 7.1s
345:	learn: 0.0345758	total: 3.75s	remaining: 7.08s
346:	learn: 0.0345341	total: 3.76s	remaining: 7.07s
347:	learn: 0.0345158	total: 3.76s	remaining: 7.05s
348:	learn: 0.0345031	total: 3.77s	remaining: 7.03s
349:	learn: 0.0344684	total: 3.78s	remaining: 7.01s
350:	learn: 0.0344576	total: 3.78s	remaining: 7s
351:	learn: 0.0344506	total: 3.79s	remaining: 6.98s
352:	learn: 0.0344239	total: 3.8s	remaining: 6.96s
353:	learn: 0.03440

494:	learn: 0.0315175	total: 4.9s	remaining: 5s
495:	learn: 0.0315024	total: 4.91s	remaining: 4.98s
496:	learn: 0.0314996	total: 4.91s	remaining: 4.97s
497:	learn: 0.0314820	total: 4.92s	remaining: 4.96s
498:	learn: 0.0314536	total: 4.93s	remaining: 4.95s
499:	learn: 0.0314314	total: 4.94s	remaining: 4.94s
500:	learn: 0.0314265	total: 4.94s	remaining: 4.92s
501:	learn: 0.0314183	total: 4.95s	remaining: 4.91s
502:	learn: 0.0314056	total: 4.96s	remaining: 4.9s
503:	learn: 0.0313847	total: 4.97s	remaining: 4.89s
504:	learn: 0.0313739	total: 4.97s	remaining: 4.88s
505:	learn: 0.0313679	total: 4.98s	remaining: 4.86s
506:	learn: 0.0313561	total: 4.99s	remaining: 4.85s
507:	learn: 0.0313508	total: 5s	remaining: 4.84s
508:	learn: 0.0313315	total: 5s	remaining: 4.83s
509:	learn: 0.0313171	total: 5.01s	remaining: 4.81s
510:	learn: 0.0312989	total: 5.02s	remaining: 4.8s
511:	learn: 0.0312853	total: 5.02s	remaining: 4.79s
512:	learn: 0.0312751	total: 5.03s	remaining: 4.78s
513:	learn: 0.0312664	to

666:	learn: 0.0295584	total: 6.16s	remaining: 3.07s
667:	learn: 0.0295463	total: 6.17s	remaining: 3.06s
668:	learn: 0.0295309	total: 6.17s	remaining: 3.06s
669:	learn: 0.0295177	total: 6.18s	remaining: 3.04s
670:	learn: 0.0295080	total: 6.19s	remaining: 3.04s
671:	learn: 0.0294971	total: 6.2s	remaining: 3.03s
672:	learn: 0.0294950	total: 6.21s	remaining: 3.02s
673:	learn: 0.0294923	total: 6.21s	remaining: 3s
674:	learn: 0.0294791	total: 6.22s	remaining: 3s
675:	learn: 0.0294610	total: 6.23s	remaining: 2.98s
676:	learn: 0.0294455	total: 6.24s	remaining: 2.98s
677:	learn: 0.0294319	total: 6.25s	remaining: 2.97s
678:	learn: 0.0294260	total: 6.25s	remaining: 2.96s
679:	learn: 0.0294220	total: 6.26s	remaining: 2.94s
680:	learn: 0.0294120	total: 6.27s	remaining: 2.94s
681:	learn: 0.0294103	total: 6.27s	remaining: 2.92s
682:	learn: 0.0294000	total: 6.28s	remaining: 2.91s
683:	learn: 0.0293924	total: 6.29s	remaining: 2.9s
684:	learn: 0.0293791	total: 6.29s	remaining: 2.89s
685:	learn: 0.029364

839:	learn: 0.0281706	total: 7.42s	remaining: 1.41s
840:	learn: 0.0281664	total: 7.43s	remaining: 1.4s
841:	learn: 0.0281599	total: 7.44s	remaining: 1.4s
842:	learn: 0.0281454	total: 7.45s	remaining: 1.39s
843:	learn: 0.0281372	total: 7.45s	remaining: 1.38s
844:	learn: 0.0281228	total: 7.46s	remaining: 1.37s
845:	learn: 0.0281123	total: 7.47s	remaining: 1.36s
846:	learn: 0.0281004	total: 7.48s	remaining: 1.35s
847:	learn: 0.0280948	total: 7.49s	remaining: 1.34s
848:	learn: 0.0280851	total: 7.5s	remaining: 1.33s
849:	learn: 0.0280812	total: 7.5s	remaining: 1.32s
850:	learn: 0.0280730	total: 7.51s	remaining: 1.31s
851:	learn: 0.0280671	total: 7.52s	remaining: 1.31s
852:	learn: 0.0280623	total: 7.53s	remaining: 1.3s
853:	learn: 0.0280604	total: 7.53s	remaining: 1.29s
854:	learn: 0.0280471	total: 7.54s	remaining: 1.28s
855:	learn: 0.0280446	total: 7.55s	remaining: 1.27s
856:	learn: 0.0280403	total: 7.55s	remaining: 1.26s
857:	learn: 0.0280357	total: 7.56s	remaining: 1.25s
858:	learn: 0.028

Best hyperparameters: OrderedDict([('bagging_temperature', 0.9), ('border_count', 100), ('depth', 10), ('grow_policy', 'Depthwise'), ('l2_leaf_reg', 1.0), ('learning_rate', 0.04930580338653668), ('min_data_in_leaf', 50)])
Best MSE score: 0.002602982696231585


In [30]:
best_hyperparameters = {
    'bagging_temperature': 0.9,
    'border_count':100,
    'depth': 10,
    'grow_policy': 'Depthwise',
    'l2_leaf_reg': 1.0,
    'learning_rate': 0.04930580338653668,
    'min_data_in_leaf': 50,
}


model_3_tun_2 = CatBoostRegressor(
    bagging_temperature=best_hyperparameters['bagging_temperature'],
    border_count=best_hyperparameters['border_count'],
    depth=best_hyperparameters['depth'],
    grow_policy=best_hyperparameters['grow_policy'],
    l2_leaf_reg=best_hyperparameters['l2_leaf_reg'],
    learning_rate=best_hyperparameters['learning_rate'],
    min_data_in_leaf=best_hyperparameters['min_data_in_leaf'],
    random_state=42  
)


model_3_tun_2.fit(X_train_sc, y_train_sc)


y_pred_3_tun_2 = model_3_tun_2.predict(X_test_sc)


mse = mean_squared_error(y_test_sc, y_pred_3_tun_2)
mae = mean_absolute_error(y_test_sc, y_pred_3_tun_2)
r2 = r2_score(y_test_sc, y_pred_3_tun_2)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_3_tun_2)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)

0:	learn: 0.1783573	total: 15.5ms	remaining: 15.5s
1:	learn: 0.1715346	total: 33.4ms	remaining: 16.7s
2:	learn: 0.1652404	total: 44.3ms	remaining: 14.7s
3:	learn: 0.1590798	total: 57.1ms	remaining: 14.2s
4:	learn: 0.1534345	total: 71.2ms	remaining: 14.2s
5:	learn: 0.1479502	total: 84.7ms	remaining: 14s
6:	learn: 0.1428466	total: 97.9ms	remaining: 13.9s
7:	learn: 0.1381261	total: 113ms	remaining: 14s
8:	learn: 0.1336002	total: 125ms	remaining: 13.7s
9:	learn: 0.1291443	total: 136ms	remaining: 13.5s
10:	learn: 0.1250764	total: 148ms	remaining: 13.3s
11:	learn: 0.1212791	total: 162ms	remaining: 13.3s
12:	learn: 0.1176018	total: 175ms	remaining: 13.3s
13:	learn: 0.1140326	total: 189ms	remaining: 13.3s
14:	learn: 0.1107658	total: 202ms	remaining: 13.2s
15:	learn: 0.1078463	total: 214ms	remaining: 13.1s
16:	learn: 0.1049178	total: 227ms	remaining: 13.1s
17:	learn: 0.1022863	total: 241ms	remaining: 13.1s
18:	learn: 0.0997259	total: 252ms	remaining: 13s
19:	learn: 0.0970918	total: 264ms	remain

178:	learn: 0.0420786	total: 2.16s	remaining: 9.92s
179:	learn: 0.0419883	total: 2.17s	remaining: 9.91s
180:	learn: 0.0419288	total: 2.19s	remaining: 9.9s
181:	learn: 0.0418479	total: 2.2s	remaining: 9.88s
182:	learn: 0.0417906	total: 2.21s	remaining: 9.85s
183:	learn: 0.0417117	total: 2.22s	remaining: 9.83s
184:	learn: 0.0416213	total: 2.23s	remaining: 9.81s
185:	learn: 0.0415981	total: 2.24s	remaining: 9.79s
186:	learn: 0.0415314	total: 2.25s	remaining: 9.76s
187:	learn: 0.0414796	total: 2.26s	remaining: 9.75s
188:	learn: 0.0413856	total: 2.27s	remaining: 9.73s
189:	learn: 0.0413540	total: 2.27s	remaining: 9.69s
190:	learn: 0.0413328	total: 2.28s	remaining: 9.66s
191:	learn: 0.0412913	total: 2.29s	remaining: 9.64s
192:	learn: 0.0412374	total: 2.3s	remaining: 9.61s
193:	learn: 0.0411511	total: 2.31s	remaining: 9.59s
194:	learn: 0.0410803	total: 2.32s	remaining: 9.58s
195:	learn: 0.0410432	total: 2.33s	remaining: 9.55s
196:	learn: 0.0409890	total: 2.34s	remaining: 9.53s
197:	learn: 0.0

364:	learn: 0.0341035	total: 3.62s	remaining: 6.3s
365:	learn: 0.0340551	total: 3.63s	remaining: 6.29s
366:	learn: 0.0340407	total: 3.64s	remaining: 6.28s
367:	learn: 0.0340225	total: 3.65s	remaining: 6.27s
368:	learn: 0.0339860	total: 3.66s	remaining: 6.25s
369:	learn: 0.0339667	total: 3.66s	remaining: 6.24s
370:	learn: 0.0339316	total: 3.67s	remaining: 6.22s
371:	learn: 0.0339083	total: 3.68s	remaining: 6.21s
372:	learn: 0.0338754	total: 3.69s	remaining: 6.2s
373:	learn: 0.0338466	total: 3.7s	remaining: 6.19s
374:	learn: 0.0338339	total: 3.7s	remaining: 6.17s
375:	learn: 0.0338147	total: 3.71s	remaining: 6.16s
376:	learn: 0.0337721	total: 3.72s	remaining: 6.15s
377:	learn: 0.0337317	total: 3.73s	remaining: 6.14s
378:	learn: 0.0337206	total: 3.73s	remaining: 6.12s
379:	learn: 0.0337008	total: 3.74s	remaining: 6.11s
380:	learn: 0.0336734	total: 3.75s	remaining: 6.09s
381:	learn: 0.0336685	total: 3.76s	remaining: 6.08s
382:	learn: 0.0336570	total: 3.76s	remaining: 6.06s
383:	learn: 0.03

534:	learn: 0.0309386	total: 4.86s	remaining: 4.22s
535:	learn: 0.0309265	total: 4.86s	remaining: 4.21s
536:	learn: 0.0309179	total: 4.87s	remaining: 4.2s
537:	learn: 0.0309069	total: 4.88s	remaining: 4.19s
538:	learn: 0.0308956	total: 4.89s	remaining: 4.18s
539:	learn: 0.0308787	total: 4.89s	remaining: 4.17s
540:	learn: 0.0308721	total: 4.9s	remaining: 4.16s
541:	learn: 0.0308587	total: 4.91s	remaining: 4.15s
542:	learn: 0.0308333	total: 4.92s	remaining: 4.14s
543:	learn: 0.0308154	total: 4.93s	remaining: 4.13s
544:	learn: 0.0308061	total: 4.93s	remaining: 4.12s
545:	learn: 0.0307887	total: 4.94s	remaining: 4.11s
546:	learn: 0.0307809	total: 4.95s	remaining: 4.1s
547:	learn: 0.0307594	total: 4.96s	remaining: 4.09s
548:	learn: 0.0307551	total: 4.96s	remaining: 4.08s
549:	learn: 0.0307494	total: 4.97s	remaining: 4.07s
550:	learn: 0.0307411	total: 4.98s	remaining: 4.05s
551:	learn: 0.0307363	total: 4.98s	remaining: 4.04s
552:	learn: 0.0307325	total: 4.99s	remaining: 4.03s
553:	learn: 0.0

704:	learn: 0.0292096	total: 6.11s	remaining: 2.56s
705:	learn: 0.0291958	total: 6.12s	remaining: 2.55s
706:	learn: 0.0291831	total: 6.13s	remaining: 2.54s
707:	learn: 0.0291776	total: 6.13s	remaining: 2.53s
708:	learn: 0.0291634	total: 6.14s	remaining: 2.52s
709:	learn: 0.0291583	total: 6.15s	remaining: 2.51s
710:	learn: 0.0291535	total: 6.16s	remaining: 2.5s
711:	learn: 0.0291412	total: 6.16s	remaining: 2.49s
712:	learn: 0.0291308	total: 6.17s	remaining: 2.48s
713:	learn: 0.0291222	total: 6.18s	remaining: 2.47s
714:	learn: 0.0291143	total: 6.19s	remaining: 2.46s
715:	learn: 0.0291121	total: 6.19s	remaining: 2.46s
716:	learn: 0.0291045	total: 6.2s	remaining: 2.45s
717:	learn: 0.0290931	total: 6.21s	remaining: 2.44s
718:	learn: 0.0290876	total: 6.21s	remaining: 2.43s
719:	learn: 0.0290793	total: 6.22s	remaining: 2.42s
720:	learn: 0.0290745	total: 6.23s	remaining: 2.41s
721:	learn: 0.0290653	total: 6.24s	remaining: 2.4s
722:	learn: 0.0290541	total: 6.25s	remaining: 2.39s
723:	learn: 0.0

886:	learn: 0.0278420	total: 7.37s	remaining: 939ms
887:	learn: 0.0278382	total: 7.38s	remaining: 931ms
888:	learn: 0.0278343	total: 7.38s	remaining: 922ms
889:	learn: 0.0278318	total: 7.39s	remaining: 913ms
890:	learn: 0.0278284	total: 7.39s	remaining: 905ms
891:	learn: 0.0278242	total: 7.4s	remaining: 896ms
892:	learn: 0.0278197	total: 7.41s	remaining: 888ms
893:	learn: 0.0278086	total: 7.42s	remaining: 879ms
894:	learn: 0.0278043	total: 7.42s	remaining: 871ms
895:	learn: 0.0277927	total: 7.43s	remaining: 862ms
896:	learn: 0.0277876	total: 7.44s	remaining: 854ms
897:	learn: 0.0277806	total: 7.44s	remaining: 845ms
898:	learn: 0.0277673	total: 7.45s	remaining: 837ms
899:	learn: 0.0277619	total: 7.46s	remaining: 829ms
900:	learn: 0.0277571	total: 7.46s	remaining: 820ms
901:	learn: 0.0277543	total: 7.47s	remaining: 812ms
902:	learn: 0.0277505	total: 7.47s	remaining: 803ms
903:	learn: 0.0277455	total: 7.48s	remaining: 795ms
904:	learn: 0.0277418	total: 7.49s	remaining: 786ms
905:	learn: 0

In [ ]:
best_hyperparameters = {
    'bagging_temperature': 0.39821776354008,
    'border_count':50,
    'depth': 8,
    'grow_policy': 'Depthwise',
    'l2_leaf_reg': 1.0,
    'learning_rate': 0.05254324533602761,
    'min_data_in_leaf': 12,
}


model_3_tun_2 = CatBoostRegressor(
    bagging_temperature=best_hyperparameters['bagging_temperature'],
    border_count=best_hyperparameters['border_count'],
    depth=best_hyperparameters['depth'],
    grow_policy=best_hyperparameters['grow_policy'],
    l2_leaf_reg=best_hyperparameters['l2_leaf_reg'],
    learning_rate=best_hyperparameters['learning_rate'],
    min_data_in_leaf=best_hyperparameters['min_data_in_leaf'],
    random_state=42  
)


model_3_tun_2.fit(X_train_sc, y_train_sc)


y_pred_3_tun_2 = model_3_tun_2.predict(X_test_sc)


mse = mean_squared_error(y_test_sc, y_pred_3_tun_2)
mae = mean_absolute_error(y_test_sc, y_pred_3_tun_2)
r2 = r2_score(y_test_sc, y_pred_3_tun_2)
mape = mean_absolute_percentage_error(y_test_sc, y_pred_3_tun_2)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("mape:", mape)
